In [1]:
#USER INPUT

#TRADE TERM
print("Your choice of 'Trade Term' will define the timeframe with which we will analyze stock returns")
print("Select a trade term that matches your current investment goals for collecting returns from stock investments")
print("")
print("Choose a Trade Term From These Options:")
print("1. 1Yrs (1 Year)")
print("2. 5Yrs (5 Years)")
print("3. 10Yrs (10 Years)")
print("4. 15Yrs (15 Years)")
print("5. 7Dys (7 Days)")
print("6. 21Dys (21 Days)")
print("7. 40Dys (40 Days)")
print("")
print("")
user_trade_term= input("Enter your trade term")
print("")
print("")
#RISK THRESHOLD
print("Your choice of 'Risk Threshold' will define how risky and volatile the stocks in the final portfolio will be")
print("Select a risk threshold that matches your comfort levels when it comes to risk")
print("")
print("Choose a Risk Threshold:")
print("1. high")
print("2. mid")
print("3. low")
print("")
print("")
user_risk= input("Enter your risk threshold")

Your choice of 'Trade Term' will define the timeframe with which we will analyze stock returns
Select a trade term that matches your current investment goals for collecting returns from stock investments

Choose a Trade Term From These Options:
1. 1Yrs (1 Year)
2. 5Yrs (5 Years)
3. 10Yrs (10 Years)
4. 15Yrs (15 Years)
5. 7Dys (7 Days)
6. 21Dys (21 Days)
7. 40Dys (40 Days)




Enter your trade term 15Yrs




Your choice of 'Risk Threshold' will define how risky and volatile the stocks in the final portfolio will be
Select a risk threshold that matches your comfort levels when it comes to risk

Choose a Risk Threshold:
1. high
2. mid
3. low




Enter your risk threshold low


In [2]:
#Import Packages
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import yfinance as yf
import pytz
from collections import Counter

In [3]:
#Define user variables | Personalize the final results
user_trade_term = '15Yrs'
user_risk= 'low'

In [4]:
#Create Term Splitting Logic
if user_trade_term == '15Yrs':
    trade_term = 15
    
elif user_trade_term == '10Yrs':
    trade_term = 10
    
elif user_trade_term == '5Yrs':
    trade_term = 5

elif user_trade_term == '1Yrs':
    trade_term = 1

elif user_trade_term == '21Dys':
    trade_term = 0.0575342

elif user_trade_term == '7Dys':
    trade_term = 0.0191781

elif user_trade_term == '40Dys':
    trade_term = 0.109589

else:
    print("Invalid user_trade_term value")
    
# Read in the stocks dataset to clean it
World_Stocks = pd.read_csv(Path(r"C:\Users\Oluwa\GITHUB\Bootcamp\ASSIGNMENTS\Project_Uno\Resources\World-Stock-Prices-Dataset.csv"))

#Cut dataset down to US stocks only
World_Stocks = World_Stocks[World_Stocks['Country'] == 'usa']

#Read in the bond yields dataset for later
Bond_yields = pd.read_csv(Path(r"C:\Users\Oluwa\GITHUB\Bootcamp\ASSIGNMENTS\Project_Uno\Resources\bond_yields_all.csv"))
                         
# Specify the format of the 'Date' column and save it as a variable
date_format = '%Y-%m-%d %H:%M:%S%z'

# Use the variable to reformat the column using a universal timezone and handle errors
try:
    World_Stocks['Date'] = pd.to_datetime(World_Stocks['Date'], format=date_format, utc=True, errors='coerce')
#Use the except functionality to print an error if this doesnt work
except Exception as e:
    print(f"An error occurred while parsing datetime values: {e}")

#Make sure Bond_yields has the same format
Bond_yields['Date'] = pd.to_datetime(Bond_yields['date'], format=date_format, utc=True, errors='coerce')

# Set 'Date' format to match 'World_Stocks' and fill missing values with 0
Bond_yields['Date'] = Bond_yields['Date'].dt.strftime('%Y-%m-%d %H:%M:%S%z')
Bond_yields.fillna(0, inplace=True)

Yr5_Bond=Bond_yields.drop(columns='date') #cleanup

# Convert the 'Date' column to datetime
Yr5_Bond['Date'] = pd.to_datetime(Yr5_Bond['Date'], format='%Y-%m-%d %H:%M:%S%z', errors='coerce')
Yr5_Bond.fillna(0, inplace=True)

# Make sure all stock data calls cut off at the same datetime by making their timezones match
cutoff_date = datetime(2023, 9, 20) - timedelta(days=365 * trade_term)

# Convert cutoff_date to a string with timezone information
cutoff_date_str = cutoff_date.strftime(date_format)

# Manually add UTC offset to the string (+00:00 for UTC) | Ensure that all dates in the string match the UTC format
cutoff_date_str += '+00:00'

# Parse the string back to a datetime64 object with the same timezone
try:
    cutoff_date = pd.to_datetime(cutoff_date_str, format=date_format, utc=True)
#Use the except functionality to print an error if this doesnt work
except Exception as e:
    print(f"An error occurred while parsing the cutoff_date: {e}")

#Replace the timezone information with the one from the first date of the dataframe. Normalizing all the date data.
cutoff_date = cutoff_date.replace(tzinfo=World_Stocks['Date'].iloc[0].tzinfo)

# Create a new dataframe variable with only the cells that are after the cut off date
Smaller_Dataframe = World_Stocks[World_Stocks['Date'] >= cutoff_date]

# Convert the 'Date' column in 'Yr5_Bond' to string format
Yr5_Bond['Date'] = Yr5_Bond['Date'].dt.strftime('%Y-%m-%d')

# Convert the 'Date' column in Smaller_Dataframe to string format
Smaller_Dataframe['Date'] = Smaller_Dataframe['Date'].astype(str)

# Extract the first 10 characters from the 'Date' column in 'Smaller_Dataframe'
matching_dates = Smaller_Dataframe['Date'].str[:10].unique()

# Filter 'Yr5_Bond' based on matching dates
Bond_yields_filtered = Yr5_Bond[Yr5_Bond['Date'].str[:10].isin(matching_dates)]

# Convert 'Date' column in 'Yr5_Bond' back to the original datetime format
Yr5_Bond['Date'] = pd.to_datetime(Yr5_Bond['Date'], format='%Y-%m-%d')

# Convert 'Date' column in 'Bond_yields_filtered' back to the original datetime format
Bond_yields_filtered['Date'] = pd.to_datetime(Bond_yields_filtered['Date'], format='%Y-%m-%d')

#For some reason I couldnt organize the dataframe unless I explicitly copied it. Tried to just make it a variable
Calculations_df = Smaller_Dataframe.copy()

#Sort the dataframe and clean the data
Calculations_df.sort_values(['Ticker', 'Date'], inplace=True)
Useless_Columns=['Dividends', 'Stock Splits']
Calculations_df.drop(columns=Useless_Columns)
Calculations_df.dropna()
Compared_Calcs= Calculations_df.groupby(['Date', 'Ticker'])

C:\Users\Oluwa\AppData\Local\Temp\ipykernel_5016\678915812.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Smaller_Dataframe['Date'] = Smaller_Dataframe['Date'].astype(str)


In [5]:
# Import data for S&P 50 from Yahoo
ticker_symbols = '^GSPC'

#Set variables for later
end_date = datetime(2023, 9, 20)
end_date = end_date.replace(tzinfo=pytz.UTC)
start_date = end_date - timedelta(days=365 * trade_term)
SPData=pd.DataFrame()

# Fetch historical data from Yahoo Finance
company = yf.Ticker(ticker_symbols)
historical_data = company.history(period="max")

# Filter data for the specified date range (5 years from the end of September)
historical_data = historical_data[(historical_data.index >= start_date) & (historical_data.index <= end_date)]

# Reset the index to make the date a column
historical_data.reset_index(inplace=True)

historical_data['Ticker'] = ticker_symbols

# Create a new DataFrame by concatenating filtered data
SPData = pd.concat([SPData, historical_data])

# Drop columns from the new DataFrame
SPData = SPData.drop(columns=Useless_Columns)

# Add new columns
SPData['Brand_Name'] = 'SP500'
SPData['Industry_Tag'] = 'Market Reference'
SPData['Country'] = 'Global'

# Drop rows with missing values (if needed)
SPData = SPData.dropna()

# Sort the DataFrame
SPData.sort_values(['Ticker', 'Date'], inplace=True)

# Initialize an empty DataFrameand list to store Beta values
beta_df = pd.DataFrame(columns=['Ticker', 'Beta'])
beta_data=[]


In [6]:
# Calculate daily returns using the Open and Close columns
Calculations_df['Daily_Return'] = Calculations_df['Close'].pct_change() * 100
SPData['SP500_Daily_Return'] = SPData['Close'].pct_change() * 100


# Create a new DataFrame for Compared_Calcs with 'Daily_Return' values
Compared_Calcs = Calculations_df[['Ticker', 'Daily_Return', 'Date']].copy()

# Check for missing values in 'Daily_Return' column of Compared_Calcs
missing_values = Compared_Calcs['Daily_Return'].isna().sum()

# Handle missing values (e.g., fill or drop them)
if missing_values > 0:
    Compared_Calcs['Daily_Return'].fillna(0, inplace=True)

#Calculate cumulative daily returns for each Ticker
Compared_Calcs['Cumulative_Return'] = (1 + Compared_Calcs['Daily_Return']).groupby(Compared_Calcs['Ticker']).cumprod()

#Calculate the average annualized yield
days_per_year = 252*trade_term  
Compared_Calcs['Yield'] = (Compared_Calcs['Cumulative_Return'] ** (1 / days_per_year) - 1) * 100

In [7]:
# Calculate Volatility
stock_vol = Calculations_df.groupby('Ticker')['Daily_Return'].std()

# Calculate Value at Risk on each section grouped by Ticker/ Date
def calculate_var(group):
    return group['Daily_Return'].quantile(1 - .95)

# Apply the function to each group
var_df = Compared_Calcs.groupby('Ticker').apply(calculate_var).reset_index()

# Create a new dataframe with this info
var_df.columns = ['Ticker', 'VaR']

# Merge the stock data and S&P 500 data based on the index
merged_data = Compared_Calcs.merge(SPData[['SP500_Daily_Return']], left_index=True, right_index=True, how='inner')

# Calculate Beta for each Ticker
for ticker in Compared_Calcs['Ticker'].unique():
    if ticker != '^GSPC':
        # Filter data for the current stock and ^GSPC
        stock_data = merged_data[merged_data['Ticker'] == ticker]
        
        # Calculate the covariance between stock returns and ^GSPC returns
        covariance = np.cov(stock_data['Daily_Return'], stock_data['SP500_Daily_Return'])[0, 1]
        
        # Calculate the variance of ^GSPC returns
        variance_SP500 = np.var(stock_data['SP500_Daily_Return'])
        
        # Calculate Beta
        beta = covariance / variance_SP500

        # Append the data as a tuple to the list
        beta_data.append((ticker, beta))
        

# Convert the list of tuples into a DataFrame
beta_df = pd.DataFrame(beta_data, columns=['Ticker', 'Beta'])

#Create a variable for the average bond yield ion a given period
Avg_5Yr_Bond_Yield = Bond_yields_filtered['CDN.AVG.3YTO5Y.AVG'].mean()

#Calculate the difference between the average bond yield and each stock's yield:
Compared_Calcs['Bond Safety Ratio'] = abs(Compared_Calcs['Yield'] - Avg_5Yr_Bond_Yield)

#Reorganize Compared Calcs by Ticker
regrouped_compared_calcs = Compared_Calcs.groupby(['Ticker', 'Date']).mean()

In [8]:
Bond_yields_filtered

,CDN.AVG.3YTO5Y.AVG,Date
0,2.30,2018-09-21
1,2.31,2018-09-21
2,2.32,2018-09-21
3,2.29,2018-09-21
4,2.29,2018-09-21
...,...,...
1242,4.04,2018-09-21
1243,4.09,2018-09-21
1244,4.12,2018-09-21
1245,4.26,2018-09-21


In [9]:
#Check for correlations
correlation_matrix1 = regrouped_compared_calcs[['Bond Safety Ratio', 'Daily_Return']].corr()
correlation_matrix2 = merged_data[['SP500_Daily_Return', 'Daily_Return']].corr()
#Both have weak correlations | Abandoning correlation matrix

In [10]:
#Drop Null Values
regrouped_compared_calcs.dropna()
beta_df.dropna()
var_df.dropna()
stock_vol.dropna()
SPData.dropna()

# Find the top 50 highest cumulative returns while excluding 'inf' values
top_cumulative_returns = regrouped_compared_calcs[regrouped_compared_calcs['Cumulative_Return'] != float('inf')]['Cumulative_Return'].nlargest(50).dropna()
# Count occurrences of each 'Ticker'
cumulative_ticker_count = top_cumulative_returns.groupby(level=0).size()
cumulative_ticker_count = cumulative_ticker_count.sort_values(ascending=False)
cumulative_ticker_count_list = cumulative_ticker_count.index.tolist()
# Find the fifteen most frequent 'Ticker' values
most_frequent_tickers_list = cumulative_ticker_count.nlargest(15).index.tolist()


# Find the best stocks based on Beta
lowest_risk_stocks = beta_df.sort_values(by='Beta').head(15).dropna()
lowest_risk_stocks_list = lowest_risk_stocks['Ticker'].tolist()

# Find the top 15 stocks with the lowest VaR
lowest_var_stock = var_df.nsmallest(15, 'VaR').dropna()
lowest_var_stock_list = lowest_var_stock['Ticker'].tolist()

# Get the 15 cells with the lowest values
lowest_volatility = stock_vol.nsmallest(15).dropna()
lowest_volatility_list = lowest_volatility.index.tolist()

# Combine all the lists into one
low_risk_analysis_list = (
    lowest_volatility_list +
    lowest_var_stock_list +
    lowest_risk_stocks_list +
    most_frequent_tickers_list
)

# Create a dictionary to count the frequency of each unique value
lr_stock_count = {}
for stock in low_risk_analysis_list:
    if stock in lr_stock_count:
        lr_stock_count[stock] += 1
    else:
        lr_stock_count[stock] = 1

# Sort the stocks by frequency (most frequent first)
low_risk_stocks_master_list = sorted(lr_stock_count.keys(), key=lambda x: lr_stock_count[x], reverse=True)



In [11]:
Yr5_Bond

,CDN.AVG.3YTO5Y.AVG,Date
0,2.30,2018-09-21
1,2.31,2018-09-21
2,2.32,2018-09-21
3,2.29,2018-09-21
4,2.29,2018-09-21
...,...,...
1242,4.04,2018-09-21
1243,4.09,2018-09-21
1244,4.12,2018-09-21
1245,4.26,2018-09-21


In [12]:
# Find the 50 lowest cumulative returns while excluding 'inf' values
worst_cumulative_returns = regrouped_compared_calcs[regrouped_compared_calcs['Cumulative_Return'] != float('inf')]['Cumulative_Return'].nsmallest(50).dropna()
# Count occurrences of each 'Ticker'
cumulative_ticker_count2 = worst_cumulative_returns.groupby(level=0).size()
cumulative_ticker_count2 = cumulative_ticker_count2.sort_values(ascending=False)
cumulative_ticker_count_list2 = cumulative_ticker_count2.index.tolist()
# Find the fifteen most frequent 'Ticker' values
most_frequent_tickers_list2 = cumulative_ticker_count2.nlargest(15).index.tolist()


# Find the worst stocks based on Beta
highest_risk_stocks = beta_df.sort_values(by='Beta').tail(15).dropna()
highest_risk_stocks_list = highest_risk_stocks['Ticker'].tolist()

# Find the 15 stocks with the lowest VaR
highest_var_stock = var_df.nlargest(15, 'VaR').dropna()
highest_var_stock_list = highest_var_stock['Ticker'].tolist()

# Get the 15 cells with the lowest volatility values
highest_volatility = stock_vol.nlargest(15).dropna()
highest_volatility_list = highest_volatility.index.tolist()

# Combine all the lists into one
high_risk_analysis_list = (
    highest_volatility_list +
    highest_var_stock_list +
    highest_risk_stocks_list +
    most_frequent_tickers_list2 
)

# Create a dictionary to count the frequency of each unique value
hr_stock_count = {}
for stock in high_risk_analysis_list:
    if stock in hr_stock_count:
        hr_stock_count[stock] += 1
    else:
        hr_stock_count[stock] = 1

# Sort the stocks by frequency (most frequent first)
high_risk_stocks_master_list = sorted(hr_stock_count.keys(), key=lambda x: hr_stock_count[x], reverse=True)


In [13]:
# Set a variable for the average risk by beta
rsk_avg = beta_df['Beta'].mean()

# Get the rows that are within an acceptable range of the average beta
average_risk_stocks = beta_df[(beta_df['Beta'] >= rsk_avg - .1) & (beta_df['Beta'] <= rsk_avg + .1)].dropna()

# Get the list of tickers for the stocks with average risk
average_risk_stocks_list = average_risk_stocks['Ticker'].tolist()

# Sort the stocks in average_risk_stocks_list by proximity to the average
average_risk_analysis_list = sorted(average_risk_stocks_list, key=lambda x: abs(beta_df.loc[beta_df['Ticker'] == x, 'Beta'].values[0] - rsk_avg))


  
# Set a variable for average cumulative return
cum_avg_ret=regrouped_compared_calcs['Cumulative_Return'].groupby('Ticker').mean().dropna()

# Calculate the average return
avg_return = cum_avg_ret.mean()  

# Filter the series by proximity to the average. Find the values that are within 1.0 of the average
average_risk_returns_list1 = cum_avg_ret[(cum_avg_ret >= avg_return - 1)].index.tolist()
average_risk_returns_list2 = cum_avg_ret[(cum_avg_ret <= avg_return + 1)].index.tolist()

# Sort both lists by proximity to avg_return
sorted_list1 = sorted(average_risk_returns_list1, key=lambda x: abs(cum_avg_ret[x] - avg_return))
sorted_list2 = sorted(average_risk_returns_list2, key=lambda x: abs(cum_avg_ret[x] - avg_return))

average_risk_returns_list = (sorted_list1 + sorted_list2)

# Sort the stocks in average_risk_returns_list by proximity to the average risk based on cumulative returns
average_risk_returns_master_list = sorted(average_risk_returns_list, key=lambda x: abs(cum_avg_ret[x] - avg_return))



# Creat the average Value At Risk Variable
var_df['VaR'] = var_df['VaR'].astype(float)
avg_var = var_df['VaR'].mean()

# Calculate the absolute difference between each VaR value and the average VaR
var_difference = var_df['VaR'].sub(avg_var).abs()

# Sort the VaR values by proximity to the average VaR
sorted_var_list = var_df['Ticker'].tolist()
sorted_var_list.sort(key=lambda x: var_difference[var_df.loc[var_df['Ticker'] == x, 'VaR'].index[0]])

# Get the top 15 values closest to the average VaR
top_15_closest_to_avg_var = sorted_var_list[:15]




# Calculate the average volatility
avg_volatility = stock_vol.mean()

# Calculate the absolute difference between each stock's volatility and the average volatility
volatility_difference = stock_vol.sub(avg_volatility).abs()

# Sort the list of stocks by proximity to the average volatility
sorted_volatility_list = stock_vol.index.tolist()
sorted_volatility_list.sort(key=lambda x: volatility_difference[x])

# Get the top 15 stocks closest to the average volatility
top_15_closest_to_avg_volatility = sorted_volatility_list[:15]



#Combine Lists
mid_combined_list= (top_15_closest_to_avg_var + top_15_closest_to_avg_volatility + average_risk_returns_master_list + average_risk_analysis_list)
mid_count= Counter(mid_combined_list)
mid_risk_stocks_master_list = [item[0] for item in sorted(mid_count.items(), key=lambda x: x[1], reverse=True)]

In [14]:
#Define the function to show a recommendaiton of 10 stocks
def print_first_10_strings(input_list):
    return input_list[:10]
    
        
#Define Final Recommendation
if user_risk == 'high':
    risk_tickers = print_first_10_strings(high_risk_stocks_master_list)
elif user_risk == 'low':
    risk_tickers = print_first_10_strings(low_risk_stocks_master_list)
elif user_risk == 'mid':
    risk_tickers = print_first_10_strings(mid_risk_stocks_master_list)
else:
    print("Invalid user_risk value")


In [15]:
#Display Final Recommendations With All Data

# Filter DataFrames by matching tickers
var_df_filtered = var_df[var_df['Ticker'].isin(risk_tickers)]
beta_df_filtered = beta_df[beta_df['Ticker'].isin(risk_tickers)]
stock_vol_filtered = stock_vol[stock_vol.index.isin(risk_tickers)]


In [16]:
#Make the volatility data a dataframe
stock_vol_filtered = stock_vol_filtered.to_frame()

# Rename the Volatility Column
stock_vol_filtered.columns = ['Volatility']

# Merge the DataFrames
merged_df = var_df_filtered.merge(beta_df_filtered, on='Ticker', how='left')

# Now, 'merged_df' contains 'Value At Risk', 'Beta', and 'Volatility' columns with 'Ticker' as the index
merged_df = merged_df.set_index('Ticker')

# Combine 'merged_df' with 'stock_vol_filtered' by the common index 'Ticker'
risk_analyzer = merged_df.join(stock_vol_filtered)

# Rename the columns to match your desired column names
risk_analyzer.rename(columns={'VaR': 'Value At Risk'}, inplace=True)

# Create a custom sorting order based on 'risk_tickers'
custom_order = {ticker: order for order, ticker in enumerate(risk_tickers)}


# Use the custom order to sort the DataFrame
risk_analyzer['sorting_order'] = risk_analyzer.index.map(custom_order)
risk_analyzer.sort_values(by='sorting_order', inplace=True)
risk_analyzer.drop('sorting_order', axis=1, inplace=True)


In [17]:
#Organize the original World_Data dataset by the final recommended list

#Remove useless columns
Useless_Columns2=['Dividends', 'Stock Splits', 'Industry_Tag', 'Country']
Final_Stock_Display= World_Stocks.drop(columns=Useless_Columns2)

#Cut the Date data down to only the daily
Final_Stock_Display['Date'] = Final_Stock_Display['Date'].dt.date

#Cut the Data down to only tickers that match the final recommended list
Final_Stock_Display=Final_Stock_Display[Final_Stock_Display['Ticker'].isin(risk_tickers)]

In [18]:
#Print Final Recommendation
risk_tickers

['PINS', 'TSLA', 'CROX', 'JWN', 'NVDA', 'DIS', 'MMM', 'PTON', 'ZI', 'AMD']

In [19]:
#Show Final Dataset
risk_analyzer

,Value At Risk,Beta,Volatility
Ticker,,,
PINS,-5.718185,-0.435202,4.949007
TSLA,-5.125984,-0.456027,3.984862
CROX,-5.119621,-0.285014,4.407196
JWN,-4.714676,-0.897449,3.497020
NVDA,-4.456210,-0.429040,3.415902
DIS,-2.681459,-0.103815,1.906029
MMM,-2.365506,-1.011772,2.045606
PTON,-7.916535,NaN,5.374018
ZI,-6.146155,-0.257212,4.975552


In [20]:
#Display World Data for Final Recommendations
Final_Stock_Display

,Date,Open,High,Low,Close,Volume,Brand_Name,Ticker
0,2023-09-20,4.840000,4.910000,4.630000,4.670000,7441900.0,peloton,PTON
10,2023-09-20,82.029999,83.199997,81.639999,82.559998,12715200.0,the walt disney company,DIS
17,2023-09-20,267.040009,273.929993,262.459991,262.589996,122514600.0,tesla,TSLA
19,2023-09-20,436.000000,439.029999,422.230011,422.390015,36710800.0,nvidia,NVDA
27,2023-09-20,27.900000,28.309999,26.820000,27.020000,40018600.0,pinterest,PINS
...,...,...,...,...,...,...,...,...
267836,2023-08-29,15.550000,15.790000,15.440000,15.670000,6393800.0,nordstrom,JWN
267837,2023-08-30,15.600000,15.870000,15.540000,15.690000,5777300.0,nordstrom,JWN
267838,2023-08-31,15.760000,16.250000,15.680000,16.219999,4982800.0,nordstrom,JWN
267839,2023-09-01,16.320000,16.530001,16.010000,16.070000,3604400.0,nordstrom,JWN
